In [ ]:
import pyap
import geometryFxns
import folium
from geometryFxns import create_office_geodataframe


xompass_gdf = create_office_geodataframe("https://www.compass.com/about/offices/")


In [ ]:
## To impliment into geometryFxns.py to ensure more addresses are geocoded
print("number of unreadable addresses is:", len(xompass_gdf[xompass_gdf['geometry'] == None]))


for index, item in enumerate(xompass_gdf['geometry']):
    if item == None:
        office_location = xompass_gdf['office_location'][index]
        address = pyap.parse(office_location, country='US')
        if len(address) > 0:
            address_dict = address[0].as_dict()
            codeable_address = str(address_dict['street_number'] + ' ' + address_dict['street_name'] + ' ' + address_dict['street_type'] + ' ' + address_dict['city'] + ', ' + address_dict['region1'] + ' ' + address_dict['postal_code'])
            print(codeable_address)
            xompass_gdf.loc[index, 'geometry'] = geometryFxns.geocode_address(codeable_address)


        

In [ ]:
print('number of unreadable addresses is:', len(xompass_gdf[xompass_gdf['geometry'] == None]))

In [ ]:


def create_map_with_geoms(gdf):
    
    #df = create_office_geodataframe(url)
    # Create a map centered on the mean of all points
    no_missing_gdf = gdf[gdf['geometry'] != None]
    points = [[point.xy[1][0], point.xy[0][0]] for point in no_missing_gdf['geometry']]
    center_lat = sum(points[0][0] for point in points) / len(no_missing_gdf)
    center_lon = sum(points[0][1] for point in points) / len(no_missing_gdf)
    
    m = folium.Map(location=[center_lat, center_lon], zoom_start=4)
    
    # Add each geometry to the map
    for i in range(len(no_missing_gdf)):
        row = no_missing_gdf.iloc[i]
        geom = row['geometry']
        folium.Marker(
            points[i],

            popup=f"{row['office_title']}<br>{row['office_location']}<br>{row['office_phone_number']}"
        ).add_to(m)
    
    filepath = 'map.html'
    m.save(filepath)
    return 

create_map_with_geoms(xompass_gdf)
